In [10]:
import pandas as pd
from docx import Document
from docx.shared import Pt
from docx.enum.text import WD_ALIGN_PARAGRAPH
import os
from rdkit import Chem
from rdkit.Chem import Descriptors, rdMolDescriptors
from datetime import datetime, timedelta

# Cargar archivo
df = pd.read_csv("BBBP.csv")
df_subset = df.head(250)

# Ruta de salida
output_dir = "informes_BBBP"
os.makedirs(output_dir, exist_ok=True)

# Fecha de mañana
fecha_manana = (datetime.now() + timedelta(days=1)).strftime("%d/%m/%Y")

def interpretar_p_np(valor):
    return "Penetra la barrera hematoencefálica (BBB)" if valor == 1 else "No penetra la barrera hematoencefálica (BBB)"

def obtener_propiedades(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        peso = Descriptors.MolWt(mol)
        formula = rdMolDescriptors.CalcMolFormula(mol)
        return peso, formula
    return None, None

def agregar_linea(doc, titulo, contenido):
    p = doc.add_paragraph()
    rt = p.add_run(titulo)
    rt.bold = True
    rt.font.name = 'Times New Roman'
    rt.font.size = Pt(12)
    rc = p.add_run(contenido)
    rc.font.name = 'Times New Roman'
    rc.font.size = Pt(12)

for idx, row in df_subset.iterrows():
    doc = Document()
    section = doc.sections[0]

    # Encabezado izquierda: Universidad
    header = section.header
    p1 = header.paragraphs[0]
    p1.text = "Universidad Privada Boliviana"
    p1.alignment = WD_ALIGN_PARAGRAPH.LEFT
    run1 = p1.runs[0]
    run1.font.name = 'Times New Roman'
    run1.font.size = Pt(10)

    # Encabezado derecha: Autor
    p2 = header.add_paragraph()
    p2.text = "Informe realizado por: Ing. Cardenas Luna Baldric"
    p2.alignment = WD_ALIGN_PARAGRAPH.RIGHT
    run2 = p2.runs[0]
    run2.font.name = 'Times New Roman'
    run2.font.size = Pt(10)

    # Título
    t = doc.add_paragraph()
    rt = t.add_run(f'Informe del Compuesto #{row["num"]}')
    rt.bold = True
    rt.underline = True
    rt.font.name = 'Times New Roman'
    rt.font.size = Pt(15)
    t.alignment = WD_ALIGN_PARAGRAPH.CENTER

    # Cuerpo
    agregar_linea(doc, "Nombre del Compuesto: ", row["name"])
    agregar_linea(doc, "Identificador Numérico: ", str(row["num"]))
    agregar_linea(doc, "Clasificación BBB: ", interpretar_p_np(row["p_np"]))
    agregar_linea(doc, "Representación SMILES: ", row["smiles"])

    peso, formula = obtener_propiedades(row["smiles"])
    if peso and formula:
        agregar_linea(doc, "Peso Molecular: ", f"{peso:.2f} g/mol")
        agregar_linea(doc, "Fórmula Molecular: ", formula)
    else:
        doc.add_paragraph("Error al interpretar SMILES.")

    # Pie de página con fecha
    footer = section.footer
    fp = footer.paragraphs[0]
    fp.text = fecha_manana
    fp.alignment = WD_ALIGN_PARAGRAPH.LEFT
    runf = fp.runs[0]
    runf.font.name = 'Times New Roman'
    runf.font.size = Pt(10)

    # Guardar
    filename = os.path.join(output_dir, f'informe_{row["num"]}.docx')
    doc.save(filename)



[22:26:57] Explicit valence for atom # 1 N, 4, is greater than permitted
[22:26:57] WARNING: not removing hydrogen atom without neighbors
[22:26:57] Explicit valence for atom # 6 N, 4, is greater than permitted
[22:26:58] WARNING: not removing hydrogen atom without neighbors
[22:26:59] WARNING: not removing hydrogen atom without neighbors
[22:26:59] WARNING: not removing hydrogen atom without neighbors
